In [1]:
import sys
import os
os.chdir("/home/jonfrey/ASL")

import numpy as np
from ucdr.visu import Visualizer
from ucdr.utils import LabelLoaderAuto


import imageio


# STD
import os
import copy

# MISC
import numpy as np
import torch
import imageio
import cv2
from PIL import Image

# matplotlib
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.backends.backend_agg import FigureCanvasAgg
import matplotlib.patches as patches

from PIL import Image
import cv2
import torch
from torchvision import transforms as tf

import numpy as np
from pathlib import Path
import copy
import numpy as np
import pickle


lla = LabelLoaderAuto(root_scannet="/home/jonfrey/Datasets/scannet")

def get_labels(p,sub=1,la="png"):
    p = [str(s) for s in Path(p).rglob(f'*.{la}') if str(s).find("_.png") == -1]
    
    p.sort(
        key=lambda x: int(x.split("/")[-1][:-4])
    )
    
    return p[::sub]

# def get_labels_multiple( base_name, scenes_list,sub=1, test=False, train=False):
#     labels = []
#     for s in scenes_list:
#         labels += get_labels(base_name.replace("xxxx",s) ,sub=sub)
#     
#     if test: 
#         return labels[int( len(labels)*0.8):]
#     
#     if train:
#         return labels[:int( len(labels)*0.8)]
#     
#     return labels


def get_labels_multiple( base_name, scenes_list,sub=1, test=False, train=False):
    labels = []
    for s in scenes_list:
        label = get_labels(base_name.replace("xxxx",s) ,sub=sub)
        if test: 
            label = label[int( len(label)*0.8):]
        elif train:
            label = label[:int( len(label)*0.8)]
        
        labels += label
    
    
    return labels

class AccMonitor():
    def __init__(self):
        self.metrics = {}
        self.metric_dict = {'correct': 0,
                            'correct_valid': 0,
                            'correct_valid_both': 0,
                            'total': 0, 
                            'total_valid':0,
                            'total_valid_both':0,
                            'images': 0}
    def register(self,name):
        self.metrics[name] = copy.deepcopy(self.metric_dict)
        
    def update(self,name, label, gt):
        if name not in self.metrics.keys():
            self.register(name)
        m1 = gt != -1
        self.metrics[name]["total_valid"] += (m1).sum()
        self.metrics[name]["correct_valid"] += np.sum(gt[m1] == label[m1])
        
        m2 = (gt != -1 )*(label != -1)
        self.metrics[name]["total_valid_both"] += (m2).sum()
        self.metrics[name]["correct_valid_both"] += np.sum(gt[m2] == label[m2])
        
        self.metrics[name]["total"] += gt.size
        self.metrics[name]["correct"] += np.sum(gt == label)
        self.metrics[name]["images"] += 1
    
    def store_pickle(self, p):
        with open(p, 'wb') as handle:
            pickle.dump(self.metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def restore_pickle(self, p):
        with open(p, 'rb') as handle:
            self.metrics = pickle.load(handle)
            
    def __str__(self):
        s = "Acc-Monitor:\n"
        for k in self.metrics.keys():
            acc = self.metrics[k]['correct_valid'] / self.metrics[k]['total_valid']
            acc2 = self.metrics[k]['correct_valid_both'] / self.metrics[k]['total_valid_both']
            s += f"  {k}: Nr-Images: {self.metrics[k]['images']} , Avg-Acc: {acc}  Avg-Acc: valid both {acc2} \n"  
        return s

In [2]:
import torch
import numpy as np
import os

from sklearn.metrics import confusion_matrix

def nanmean(data, **args):
    # This makes it ignore the first 'background' class
    return np.ma.masked_array(data, np.isnan(data)).mean(**args)
    # In np.ma.masked_array(data, np.isnan(data), elements of data == np.nan is invalid and will be ingorned during computation of np.mean()


class SemanticsMeter:
    def __init__(self, number_classes):
        self.conf_mat = None
        self.number_classes = number_classes

    def clear(self):
        self.conf_mat = None

    def prepare_inputs(self, *inputs):
        outputs = []
        for i, inp in enumerate(inputs):
            if torch.is_tensor(inp):
                inp = inp.detach().cpu().numpy()
            outputs.append(inp)

        return outputs

    def update(self, preds, truths):
        preds, truths = self.prepare_inputs(preds, truths) # [B, N, 3] or [B, H, W, 3], range[0, 1]
        preds = preds.flatten()
        truths = truths.flatten()
        valid_pix_ids = truths!=-1
        preds = preds[valid_pix_ids] 
        truths = truths[valid_pix_ids]
        conf_mat_current = confusion_matrix(truths, preds, labels=list(range(self.number_classes)))
        if self.conf_mat is None:
            self.conf_mat = conf_mat_current
        else:
            self.conf_mat += conf_mat_current

    def measure(self):
        conf_mat = self.conf_mat
        norm_conf_mat = np.transpose(
            np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))

        missing_class_mask = np.isnan(norm_conf_mat.sum(1)) # missing class will have NaN at corresponding class
        exsiting_class_mask = ~ missing_class_mask

        class_average_accuracy = nanmean(np.diagonal(norm_conf_mat))
        total_accuracy = (np.sum(np.diagonal(conf_mat)) / np.sum(conf_mat))
        ious = np.zeros(self.number_classes)
        for class_id in range(self.number_classes):
            ious[class_id] = (conf_mat[class_id, class_id] / (
                    np.sum(conf_mat[class_id, :]) + np.sum(conf_mat[:, class_id]) -
                    conf_mat[class_id, class_id]))
        miou_valid_class = np.mean(ious[exsiting_class_mask])
        return miou_valid_class, total_accuracy, class_average_accuracy





In [3]:
gt_p = "/home/jonfrey/Datasets/scannet/scans/xxxx/label-filt"

data = []
data.append( [ 1, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_pretrained_2/scans/xxxx", "iter1_pred"])
data.append( [ 1, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_4_scenes_map_1/scans/xxxx" ,"iter1_repro" ])
data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration2_individual_no_replay/scans/xxxx", "iter2_0_pred"])
data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration2_individual_no_replay_reprojected/scans/xxxx", "iter2_0_repro"])
data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration_2_individual_replay02/scans/xxxx", "iter2_02_pred"])
data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration_2_individual_replay02_reprojected/scans/xxxx", "iter2_02_repro"])
data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration_2_individual_replay05/scans/xxxx", "iter2_05_pred"])
data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration_2_individual_replay09/scans/xxxx", "iter2_9_pred"])

data.append( [ 2, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iteration_3_individual_replay02_reprojected/scans/xxxx" , "iter3_02_repro"])
data.append( [ 3, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iteration_3_individual_replay02/scans/xxxx" , "iter3_02_pred"])
data.append( [ 1, 5, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter1_5cm_reprojected/scans/xxxx" , "iter1_repro_5cm"])
data.append( [ 1, 5, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_gt_reprojected/scans/xxxx" , "gt_repro_5cm"])
data.append( [ 1, 3, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_gt_reprojected_3cm/scans/xxxx" , "gt_repro_3cm"])
data.append( [ 1, 5, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter2_5cm_replay00/scans/xxxx" , "iter2_00_5cm_pred"])
data.append( [ 1, 5, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter2_5cm_replay02/scans/xxxx" , "iter2_02_5cm_pred"])
data.append( [ 1, 5, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter2_5cm_rep02_reprojected/scans/xxxx" , "iter2_02_5cm_repro"])

data.append( [ 1, 5, f"/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter2_5cm_replay02_e20/scans/xxxx" , "iter2_02_20_epochs_5cm_pred"])
scenes = [["scene0000_00",
        "scene0000_01",
        "scene0000_02"],
        ["scene0001_00",
        "scene0001_01"],
        ["scene0002_00",
        "scene0002_01"],
        ["scene0003_00",
        "scene0003_01",
       "scene0003_02"],
        ["scene0004_00"],
        ["scene0005_00",
        "scene0005_01"]]

data_arr = np.array( data )


In [9]:
 data_arr

array([['1', '3',
        '/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_pretrained_2/scans/xxxx',
        'iter1_pred'],
       ['1', '3',
        '/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_4_scenes_map_1/scans/xxxx',
        'iter1_repro'],
       ['2', '3',
        '/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration2_individual_no_replay/scans/xxxx',
        'iter2_0_pred'],
       ['2', '3',
        '/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration2_individual_no_replay_reprojected/scans/xxxx',
        'iter2_0_repro'],
       ['2', '3',
        '/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration_2_individual_replay02/scans/xxxx',
        'iter2_02_pred'],
       ['2', '3',
        '/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/iteration_2_individual_replay02_reprojected/scans/xxxx',
        'iter2_02_repro'],
       

In [4]:
def array_to_acc( data_arr, gt_p, scenes , mode = "test", force=True, clean=False):
    NR = data_arr.shape[0]
    
    
    if clean: 
        for i in range(NR):
            path = data_arr[i,3]+"_"+ mode +"_res.pkl"
            try:
                os.remove(path)
            except:
                pass
    if mode == "test":
        test = True
    else:
        test = False
    
    if mode == "train":
        train = True
    else:
        train = False
    
    acc = np.zeros( (NR, len(scenes)) )
    miou = np.zeros( (NR, len(scenes)) )
    
    for i in range(NR):
        path = data_arr[i,3]+"_"+ mode +"_res.pkl"
        print(path)
        if os.path.isfile(path) and not force:
                with open(path, 'rb') as handle:
                    acc[i] = pickle.load(handle)
                
        else:
        
            for nr_s, s in enumerate( scenes):
                accm = AccMonitor() 
                sm = SemanticsMeter(number_classes=40)
                sm.clear()
                print(data_arr[i,2])
                print(s)
                gt_paths = get_labels_multiple( gt_p, s ,10, test, train)
                ppp = get_labels_multiple( data_arr[i,2] , s ,1, test, train)
                print( len( ppp ) )
                if len( ppp) != len(gt_paths):
                    print("ERROR break")
                    return False
                
                
                for j, gtp in enumerate( gt_paths):
                    gt = lla.get( gtp )[0]
                    tmp = lla.get( ppp[j] )[0]
                    accm.update("acc", tmp-1 ,gt-1)
                    sm.update(tmp-1, gt-1)
                    
                    if j % 10 == 0 :
                       print(j, "/", len(gt_paths))
                
                miou_valid_class, total_accuracy, class_average_accuracy = sm.measure()
                acc[i,nr_s] = accm.metrics["acc"]['correct_valid'] / accm.metrics["acc"]['total_valid']
                miou[i,nr_s] = miou_valid_class
                print( "new", miou_valid_class, total_accuracy, class_average_accuracy )
                print( "old", acc[i,nr_s])
                
            with open(path.replace("pkl", "_acc.pkl"), 'wb') as handle:
                pickle.dump(acc[i], handle, protocol=pickle.HIGHEST_PROTOCOL)
                print(acc)
                
            with open(path.replace("pkl", "_iou.pkl"), 'wb') as handle:
                pickle.dump(miou[i], handle, protocol=pickle.HIGHEST_PROTOCOL)
                print(acc)

    return acc, miou

results_acc, miou = array_to_acc( data_arr, gt_p, scenes , "test", True, False)

iter1_pred_test_res.pkl
/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_pretrained_2/scans/xxxx
['scene0000_00', 'scene0000_01', 'scene0000_02']
355
0 / 355
10 / 355
20 / 355
30 / 355
40 / 355
50 / 355
60 / 355
70 / 355
80 / 355
90 / 355
100 / 355
110 / 355
120 / 355
130 / 355
140 / 355
150 / 355
160 / 355
170 / 355
180 / 355
190 / 355
200 / 355
210 / 355
220 / 355
230 / 355
240 / 355
250 / 355
260 / 355
270 / 355
280 / 355
290 / 355
300 / 355
310 / 355
320 / 355
330 / 355
340 / 355
350 / 355


<ipython-input-2-9da0cd683057>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-9da0cd683057>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (


new 0.17598796913823178 0.6005975406654301 0.2724240468598788
old 0.6005975406654301
/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_pretrained_2/scans/xxxx
['scene0001_00', 'scene0001_01']
57
0 / 57
10 / 57
20 / 57
30 / 57
40 / 57
50 / 57


<ipython-input-2-9da0cd683057>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-9da0cd683057>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (


new 0.17485490599093428 0.4551795028164489 0.2828361797929088
old 0.4551795028164489
/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_pretrained_2/scans/xxxx
['scene0002_00', 'scene0002_01']
250
0 / 250
10 / 250
20 / 250
30 / 250
40 / 250
50 / 250
60 / 250
70 / 250
80 / 250
90 / 250
100 / 250
110 / 250
120 / 250
130 / 250
140 / 250
150 / 250
160 / 250
170 / 250
180 / 250
190 / 250
200 / 250
210 / 250
220 / 250
230 / 250
240 / 250


...

<ipython-input-2-9da0cd683057>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-9da0cd683057>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (


new 0.44506880365588064 0.7565915830469588 0.48560724166194763
old 0.7565915830469588
/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter2_5cm_replay02_e20/scans/xxxx
['scene0004_00']
19
0 / 19
10 / 19


<ipython-input-2-9da0cd683057>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-9da0cd683057>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (


new 0.27953141324947095 0.42328039291432074 0.3609669586334858
old 0.42328039291432074
/media/jonfrey/Fieldtrip1/Jonas_Frey_Master_Thesis/labels_generated/labels_iter2_5cm_replay02_e20/scans/xxxx
['scene0005_00', 'scene0005_01']
53
0 / 53
10 / 53
20 / 53
30 / 53
40 / 53
50 / 53
new 0.2182068694749036 0.50103123214539 0.2946888797754191
old 0.50103123214539
[[0.60059754 0.4551795  0.34814029 0.70522183 0.40322308 0.4678581 ]
 [0.67853572 0.53853143 0.42430488 0.79100054 0.43985224 0.49580453]
 [0.68293918 0.43341723 0.38264128 0.76567524 0.40976906 0.52315909]
 [0.69555606 0.4540859  0.40943539 0.7700388  0.42128564 0.56538546]
 [0.67707277 0.45853175 0.38652306 0.75853396 0.42671741 0.53248153]
 [0.6923635  0.48704059 0.41343683 0.76951715 0.42291923 0.57106378]
 [0.67797051 0.43651064 0.38545796 0.7421195  0.42189349 0.53852622]
 [0.64691824 0.33494868 0.37368016 0.74254984 0.40955072 0.50402478]
 [0.67954606 0.42923968 0.39567473 0.74785112 0.39109449 0.57674908]
 [0.66909258 0.47906

<ipython-input-2-9da0cd683057>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-9da0cd683057>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (


In [31]:

def get_labels(p,sub=1,la="png"):
    p = [str(s) for s in Path(p).rglob(f'*.{la}') if str(s).find("_.png") == -1]
    
    p.sort(
        key=lambda x: int(x.split("/")[-1][:-4])
    )
    
    return p[::sub]

def get_labels_multiple( base_name, scenes_list,sub=1, test=False, train=False):
    labels = []
    for s in scenes_list:
        labels += get_labels(base_name.replace("xxxx",s) ,sub=sub)
    
    if test: 
        return labels[int( len(labels)*0.8):]
    
    if train:
        return labels[:int( len(labels)*0.8)]
    
    return labels

def get_labels_multiple_cut_per_scene( base_name, scenes_list,sub=1, test=False, train=False):
    labels = []
    for s in scenes_list:
        label = get_labels(base_name.replace("xxxx",s) ,sub=sub)
        if test: 
            label = label[int( len(label)*0.8):]
        elif train:
            label = label[:int( len(label)*0.8)]
        
        labels += label
    
    
    return labels

f = lambda x: int( x.split("/")[-3][5:9])* 10000000 + int( x.split("/")[-3][10:12])* 10000 + int( x.split("/")[-1].split(".")[0])

gt_paths = get_labels_multiple( gt_p, scenes[0] ,10,False, True)
#gt_paths
gt_paths.sort(key=f)

print(gt_paths[0], gt_paths[-1], len(gt_paths))

gt_paths = get_labels_multiple( gt_p, scenes[0] ,10,True, False)
#gt_paths
gt_paths.sort(key=f)
print( gt_paths[0], gt_paths[-1], len(gt_paths))

/home/jonfrey/Datasets/scannet/scans/scene0000_00/label-filt/0.png /home/jonfrey/Datasets/scannet/scans/scene0000_02/label-filt/2620.png 1413
/home/jonfrey/Datasets/scannet/scans/scene0000_02/label-filt/2630.png /home/jonfrey/Datasets/scannet/scans/scene0000_02/label-filt/6160.png 354


In [28]:

x = gt_paths[0]
x.split("/")[-1].split(".")[0]

'2630'

In [6]:
H,W = 100, 100

sm = SemanticsMeter(number_classes=40)
# miou_valid_class, total_accuracy, class_average_accuracy 

pred = torch.ones( (H,W), dtype=torch.float32)


label= torch.ones( (H,W), dtype=torch.float32)
label[0,:] = -1
label[1:,:] = 1
# We only have a single class. The prediction is the same as the label. mIoU = 1

sm.update(pred, label)
print(sm.measure())
sm.clear()


label= torch.ones( (H,W), dtype=torch.float32)
label[0,:] = -1
label[1:50,:] = 0
# We have two classes
# class 0:
# TP = 0, -> IoU = 0

# class 1:
# TP = 50 , TP = 50 , FP = 49, FN = 0 -> IoU 50/99

# mIoU = 0 / 2 + (50/99)/2 = 0.25252525252


sm.update(pred, label)
print(sm.measure())
sm.clear()

label= torch.ones( (H,W), dtype=torch.float32)
label[:50,:] = 0
label[50:,:] = 1

sm.update(pred, label)
print(sm.measure())
sm.clear()

# We have two classes
# class 0:
# TP = 0, -> IoU = 0

# class 1:
# TP = 50 , TP = 50 , FP = 50, FN = 0 -> IoU 50/100

# mIoU = 0 / 2 + (50/100)/2 = 0.25



(1.0, 1.0, 1.0)
(0.25252525252525254, 0.5050505050505051, 0.5)
(0.25, 0.5, 0.5)


<ipython-input-2-9da0cd683057>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-9da0cd683057>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (
<ipython-input-2-9da0cd683057>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-9da0cd683057>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (
<ipython-input-2-9da0cd683057>:46: RuntimeWarning: invalid value encountered in true_divide
  np.transpose(conf_mat) / conf_mat.astype(np.float).sum(axis=1))
<ipython-input-2-9da0cd683057>:55: RuntimeWarning: invalid value encountered in long_scalars
  ious[class_id] = (conf_mat[class_id, class_id] / (


In [7]:
# FOR IOU 
import pandas as pd
names = data_arr[:,-1].tolist()
df = pd.DataFrame(data= miou[:,:-1].T, columns=names)
df.mean()

d2 = np.zeros( (miou[:,:-1].shape[0],miou[:,:-1].shape[1]+1 ) )
d2[:,:-1] = miou[:,:-1]
d2[:,-1] = df.mean()

df2 = pd.DataFrame(data= d2.T, columns=names )

print("iter_1 == pretrained network output")
print("iter_1_reprojected == pseudo label reprojected")
print("cl -> network output;  XX percentage of replayed samples \n \n")

print( df.mean() )
#print( df.std() )

df2

iter_1 == pretrained network output
iter_1_reprojected == pseudo label reprojected
cl -> network output;  XX percentage of replayed samples 
 

iter1_pred                     0.204874
iter1_repro                    0.263893
iter2_0_pred                   0.231017
iter2_0_repro                  0.247211
iter2_02_pred                  0.238284
iter2_02_repro                 0.253935
iter2_05_pred                  0.234663
iter2_9_pred                   0.206795
iter3_02_repro                 0.229823
iter3_02_pred                  0.223832
iter1_repro_5cm                0.269440
gt_repro_5cm                   0.772534
gt_repro_3cm                   0.805904
iter2_00_5cm_pred              0.227676
iter2_02_5cm_pred              0.234008
iter2_02_5cm_repro             0.252414
iter2_02_20_epochs_5cm_pred    0.225157
dtype: float64


,iter1_pred,iter1_repro,iter2_0_pred,iter2_0_repro,iter2_02_pred,iter2_02_repro,iter2_05_pred,iter2_9_pred,iter3_02_repro,iter3_02_pred,iter1_repro_5cm,gt_repro_5cm,gt_repro_3cm,iter2_00_5cm_pred,iter2_02_5cm_pred,iter2_02_5cm_repro,iter2_02_20_epochs_5cm_pred
0,0.175988,0.210875,0.192063,0.222779,0.180831,0.210759,0.188608,0.166610,0.204510,0.175844,0.224837,0.778865,0.820689,0.192077,0.180367,0.208732,0.177235
1,0.174855,0.200286,0.147424,0.153862,0.155511,0.166298,0.151419,0.118101,0.127421,0.150800,0.190903,0.760303,0.770100,0.113553,0.135513,0.161580,0.114108
2,0.119287,0.145767,0.112980,0.133094,0.113945,0.136101,0.117147,0.101536,0.117319,0.104192,0.144718,0.756392,0.820624,0.112906,0.108623,0.140469,0.109843
3,0.329452,0.489249,0.431899,0.440157,0.446979,0.461230,0.434753,0.426501,0.424908,0.417086,0.492812,0.858859,0.909720,0.428487,0.445879,0.450815,0.445069
4,0.224789,0.273287,0.270720,0.286162,0.294156,0.295284,0.281388,0.221228,0.274956,0.271235,0.293932,0.708250,0.708388,0.291357,0.299655,0.300475,0.279531
5,0.204874,0.263893,0.231017,0.247211,0.238284,0.253935,0.234663,0.206795,0.229823,0.223832,0.269440,0.772534,0.805904,0.227676,0.234008,0.252414,0.225157


In [11]:
# FOR IOU 
import pandas as pd
names = data_arr[:,-1].tolist()
df = pd.DataFrame(data= miou[:,:-1].T, columns=names)
df.mean()

d2 = np.zeros( (miou[:,:-1].shape[0],miou[:,:-1].shape[1]+1 ) )
d2[:,:-1] = miou[:,:-1]
d2[:,-1] = df.mean()

df2 = pd.DataFrame(data= d2.T, columns=names )

print("iter_1 == pretrained network output")
print("iter_1_reprojected == pseudo label reprojected")
print("cl -> network output;  XX percentage of replayed samples \n \n")

print( df.mean() )
#print( df.std() )

df2


iter_1 == pretrained network output
iter_1_reprojected == pseudo label reprojected
cl -> network output;  XX percentage of replayed samples 
 

iter1_pred                     0.204874
iter1_repro                    0.263893
iter2_0_pred                   0.231017
iter2_0_repro                  0.247211
iter2_02_pred                  0.238284
iter2_02_repro                 0.253935
iter2_05_pred                  0.234663
iter2_9_pred                   0.206795
iter3_02_repro                 0.229823
iter3_02_pred                  0.223832
iter1_repro_5cm                0.269440
gt_repro_5cm                   0.772534
gt_repro_3cm                   0.805904
iter2_00_5cm_pred              0.227676
iter2_02_5cm_pred              0.234008
iter2_02_5cm_repro             0.252414
iter2_02_20_epochs_5cm_pred    0.225157
dtype: float64


,iter1_pred,iter1_repro,iter2_0_pred,iter2_0_repro,iter2_02_pred,iter2_02_repro,iter2_05_pred,iter2_9_pred,iter3_02_repro,iter3_02_pred,iter1_repro_5cm,gt_repro_5cm,gt_repro_3cm,iter2_00_5cm_pred,iter2_02_5cm_pred,iter2_02_5cm_repro,iter2_02_20_epochs_5cm_pred
0,0.175988,0.210875,0.192063,0.222779,0.180831,0.210759,0.188608,0.166610,0.204510,0.175844,0.224837,0.778865,0.820689,0.192077,0.180367,0.208732,0.177235
1,0.174855,0.200286,0.147424,0.153862,0.155511,0.166298,0.151419,0.118101,0.127421,0.150800,0.190903,0.760303,0.770100,0.113553,0.135513,0.161580,0.114108
2,0.119287,0.145767,0.112980,0.133094,0.113945,0.136101,0.117147,0.101536,0.117319,0.104192,0.144718,0.756392,0.820624,0.112906,0.108623,0.140469,0.109843
3,0.329452,0.489249,0.431899,0.440157,0.446979,0.461230,0.434753,0.426501,0.424908,0.417086,0.492812,0.858859,0.909720,0.428487,0.445879,0.450815,0.445069
4,0.224789,0.273287,0.270720,0.286162,0.294156,0.295284,0.281388,0.221228,0.274956,0.271235,0.293932,0.708250,0.708388,0.291357,0.299655,0.300475,0.279531
5,0.204874,0.263893,0.231017,0.247211,0.238284,0.253935,0.234663,0.206795,0.229823,0.223832,0.269440,0.772534,0.805904,0.227676,0.234008,0.252414,0.225157


In [12]:
# FOR IOU
(df2*100).round(1)



,iter1_pred,iter1_repro,iter2_0_pred,iter2_0_repro,iter2_02_pred,iter2_02_repro,iter2_05_pred,iter2_9_pred,iter3_02_repro,iter3_02_pred,iter1_repro_5cm,gt_repro_5cm,gt_repro_3cm,iter2_00_5cm_pred,iter2_02_5cm_pred,iter2_02_5cm_repro,iter2_02_20_epochs_5cm_pred
0,17.6,21.1,19.2,22.3,18.1,21.1,18.9,16.7,20.5,17.6,22.5,77.9,82.1,19.2,18.0,20.9,17.7
1,17.5,20.0,14.7,15.4,15.6,16.6,15.1,11.8,12.7,15.1,19.1,76.0,77.0,11.4,13.6,16.2,11.4
2,11.9,14.6,11.3,13.3,11.4,13.6,11.7,10.2,11.7,10.4,14.5,75.6,82.1,11.3,10.9,14.0,11.0
3,32.9,48.9,43.2,44.0,44.7,46.1,43.5,42.7,42.5,41.7,49.3,85.9,91.0,42.8,44.6,45.1,44.5
4,22.5,27.3,27.1,28.6,29.4,29.5,28.1,22.1,27.5,27.1,29.4,70.8,70.8,29.1,30.0,30.0,28.0
5,20.5,26.4,23.1,24.7,23.8,25.4,23.5,20.7,23.0,22.4,26.9,77.3,80.6,22.8,23.4,25.2,22.5


In [8]:
# FOR ACC
import pandas as pd
names = data_arr[:,-1].tolist()
df = pd.DataFrame(data= results_acc[:,:-1].T, columns=names)
df.mean()

d2 = np.zeros( (results_acc[:,:-1].shape[0],results_acc[:,:-1].shape[1]+1 ) )
d2[:,:-1] = results_acc[:,:-1]
d2[:,-1] = df.mean()

df2 = pd.DataFrame(data= d2.T, columns=names )

print("iter_1 == pretrained network output")
print("iter_1_reprojected == pseudo label reprojected")
print("cl -> network output;  XX percentage of replayed samples \n \n")

print( df.mean() )
#print( df.std() )

df2

iter_1 == pretrained network output
iter_1_reprojected == pseudo label reprojected
cl -> network output;  XX percentage of replayed samples 
 

iter1_pred                     0.502472
iter1_repro                    0.574445
iter2_0_pred                   0.534888
iter2_0_repro                  0.550080
iter2_02_pred                  0.541476
iter2_02_repro                 0.557055
iter2_05_pred                  0.532790
iter2_9_pred                   0.501530
iter3_02_repro                 0.528681
iter3_02_pred                  0.533689
iter1_repro_5cm                0.571867
gt_repro_5cm                   0.924739
gt_repro_3cm                   0.935302
iter2_00_5cm_pred              0.518951
iter2_02_5cm_pred              0.531713
iter2_02_5cm_repro             0.547579
iter2_02_20_epochs_5cm_pred    0.516648
dtype: float64


,iter1_pred,iter1_repro,iter2_0_pred,iter2_0_repro,iter2_02_pred,iter2_02_repro,iter2_05_pred,iter2_9_pred,iter3_02_repro,iter3_02_pred,iter1_repro_5cm,gt_repro_5cm,gt_repro_3cm,iter2_00_5cm_pred,iter2_02_5cm_pred,iter2_02_5cm_repro,iter2_02_20_epochs_5cm_pred
0,0.600598,0.678536,0.682939,0.695556,0.677073,0.692364,0.677971,0.646918,0.679546,0.669093,0.687255,0.932262,0.951060,0.681306,0.675841,0.683970,0.672059
1,0.455180,0.538531,0.433417,0.454086,0.458532,0.487041,0.436511,0.334949,0.429240,0.479063,0.511139,0.947667,0.949459,0.344357,0.404929,0.453743,0.344568
2,0.348140,0.424305,0.382641,0.409435,0.386523,0.413437,0.385458,0.373680,0.395675,0.379095,0.419113,0.899615,0.936773,0.387420,0.388985,0.411459,0.386742
3,0.705222,0.791001,0.765675,0.770039,0.758534,0.769517,0.742120,0.742550,0.747851,0.740496,0.793968,0.956982,0.971482,0.763104,0.761839,0.765157,0.756592
4,0.403223,0.439852,0.409769,0.421286,0.426717,0.422919,0.421893,0.409551,0.391094,0.400696,0.447860,0.887168,0.867738,0.418569,0.426968,0.423566,0.423280
5,0.502472,0.574445,0.534888,0.550080,0.541476,0.557055,0.532790,0.501530,0.528681,0.533689,0.571867,0.924739,0.935302,0.518951,0.531713,0.547579,0.516648


In [10]:
(df2*100).round(1)

,iter1_pred,iter1_repro,iter2_0_pred,iter2_0_repro,iter2_02_pred,iter2_02_repro,iter2_05_pred,iter2_9_pred,iter3_02_repro,iter3_02_pred,iter1_repro_5cm,gt_repro_5cm,gt_repro_3cm,iter2_00_5cm_pred,iter2_02_5cm_pred,iter2_02_5cm_repro,iter2_02_20_epochs_5cm_pred
0,60.1,67.9,68.3,69.6,67.7,69.2,67.8,64.7,68.0,66.9,68.7,93.2,95.1,68.1,67.6,68.4,67.2
1,45.5,53.9,43.3,45.4,45.9,48.7,43.7,33.5,42.9,47.9,51.1,94.8,94.9,34.4,40.5,45.4,34.5
2,34.8,42.4,38.3,40.9,38.7,41.3,38.5,37.4,39.6,37.9,41.9,90.0,93.7,38.7,38.9,41.1,38.7
3,70.5,79.1,76.6,77.0,75.9,77.0,74.2,74.3,74.8,74.0,79.4,95.7,97.1,76.3,76.2,76.5,75.7
4,40.3,44.0,41.0,42.1,42.7,42.3,42.2,41.0,39.1,40.1,44.8,88.7,86.8,41.9,42.7,42.4,42.3
5,50.2,57.4,53.5,55.0,54.1,55.7,53.3,50.2,52.9,53.4,57.2,92.5,93.5,51.9,53.2,54.8,51.7


In [ ]:
visu = Visualizer( p_visu= "/home/jonfrey/Pictures/test")    
    
gt_paths = get_labels_multiple( gt_p, ["scene0000_00"] ,10, False,False)
ppp = get_labels_multiple( data_arr[i,2] , ["scene0000_00"] ,1, False,False)

In [ ]:
target = "labels_iter2_5cm_replay02"
base = "/home/jonfrey/Datasets/labels_generated/" + target + "/scans"

for s in [x for x in os.listdir(base)]:
    print(s)
    b =  os.path.join(base,s)
    bb = os.listdir( b)[0]
    src = os.path.join(b,bb)
    dst = os.path.join(b,target)
    print( src, dst)
    os.rename(src, dst)

In [ ]:
import shutil 
target = "iteration_2_individual_replay09"
base = "/home/jonfrey/Datasets/labels_generated/" + target
for s in [x[0] for x in os.walk(base) if x[0].split("/")[-1][:7] ==  "scene00"] :
    dst = os.path.join(base, "scans")
    
    try:
        pass
        #shutil.move(s, dst)
    except:
        pass